In [48]:
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

In [49]:
driver = webdriver.Chrome()

In [50]:
#driver.get('https://www.amazon.com/s?k=laptops&page=2&crid=KYJXXSGJUBEZ&qid=1717268206&sprefix=laptops%2Caps%2C121&ref=sr_pg_2')
driver.get('https://www.amazon.com/s?k=mobile+phones&crid=16Z7FEE6864YU&sprefix=mobile+phones%2Caps%2C141&ref=nb_sb_noss_1')

In [51]:
html_data = BeautifulSoup(driver.page_source, 'html.parser')

In [52]:
pagination_element = html_data.find('span', {'class': 's-pagination-item s-pagination-disabled'})
if pagination_element is not None:
    number_of_pages = int(pagination_element.text)
else:
    number_of_pages = 0

In [53]:
number_of_pages

20

In [54]:
products = html_data.find_all('div', {'data-component-type': 's-search-result'})

In [55]:
titles = []
images = []
ratings = []
prices = []

for i in tqdm(range(number_of_pages)):
    #url = 'https://www.amazon.com/s?k=laptops&page=2&crid=KYJXXSGJUBEZ&qid=1717268206&sprefix=laptops%2Caps%2C121&ref=sr_pg_2='+ str(i+1)
    url = 'https://www.amazon.com/s?k=mobile+phones&crid=16Z7FEE6864YU&sprefix=mobile+phones%2Caps%2C141&ref=nb_sb_noss_1&page='+ str(i+1)
    driver.get(url)
    time.sleep(3)
    html_data = BeautifulSoup(driver.page_source, 'html.parser')
    products = html_data.find_all('div', {'data-component-type': 's-search-result'})
    for product in products:
        title_element = product.find('span', {'class': 'a-size-medium a-color-base a-text-normal'}) 
        if title_element:
            title = title_element.text
            titles.append(title)
            img = product.find('img')['src']
            images.append(img)
            rating = product.find('span', {'class' : 'a-icon-alt'})
            if rating == None:
                rating = 'No Rating'
            else:
                rating = product.find('span', {'class' : 'a-icon-alt'}).text
            ratings.append(rating)
            price_whole_element = product.find('span', {'class': 'a-price-whole'})
            price_fraction_element = product.find('span', {'class': 'a-price-fraction'})
            if price_whole_element and price_fraction_element:
                price = '$' + price_whole_element.text + price_fraction_element.text
            elif price_whole_element:
                price = '$' + price_whole_element.text
            else:
                price = 'No Price'
            prices.append(price)

100%|███████████████████████████████████████████| 20/20 [01:48<00:00,  5.42s/it]


In [56]:
len(titles)

402

In [57]:
data = pd.DataFrame({'titles':titles, 'images':images, 'ratings':ratings, 'prices':prices})

In [58]:
#data.to_csv('Laptop Products.csv')
data.to_csv('Mobile Products.csv')